In [21]:
# Importing required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import requests
from datetime import datetime


In [22]:
# CONFIGURATION VARIABLES - Edit these to customize your retirement scenario


# Personal Information
year_of_birth = 1992
Age_diff_spouse = 4
retirement_year = 2028

# Market Parameters
stock_market_mean_yoy_growthrate_percent = 8
stock_market_std_yoy_growthrate = 18.0977

# Investments Portfolio
Investments = {
    'Ret_accnts': {
        'type': 'stock_retirement',
        'current_value': 324834,
        'beta': 1,
        'mean_yoy_growthrate_percent': 10.26,
        'std_yoy_growthrate': 18.0977,
        'additions_yearly': 0,
        'currency': 'USD'
    },
    'TSLA': {
        'type': 'stock',
        'current_value': 1750000,
        'beta': 1,
        'mean_yoy_growthrate_percent': 9,
        'std_yoy_growthrate': 58.0977,
        'additions_yearly': 0,
        'currency': 'USD'
    },
    'Diversiifed_stock': {
        'type': 'stock',
        'current_value': 1400000,  # Updated value
        'beta': 1,
        'mean_yoy_growthrate_percent': 10.26,
        'std_yoy_growthrate': 18.0977,
        'additions_yearly': 0,
        'currency': 'USD'
    },
    'Marcus': {
        'type': 'cash_certificate',
        'current_value': 20000,
        'additions_yearly': 0,
        'mean_yoy_growthrate_percent': 5,
        'currency': 'USD'
    }
}

# Social Security - Based on Actual Earnings History
# Actual earnings history (Taxed Social Security Earnings)
actual_earnings_history = {
    2014: 37,
    2015: 0,
    2016: 0,
    2017: 24022,
    2018: 123623,
    2019: 121380,
    2020: 137700,
    2021: 142800,
    2022: 133629,
    2023: 160200,
    2024: 168600
}

social_sec = {
    'type': 'social_security',
    'withdrawl_age': 62,
    'earnings_history': actual_earnings_history,
    'future_earnings_assumption': 168600,  # Use 2024 earnings for future years
    'avg_cola_yoy_growth': 2.6,
    'avg_cola_std': 10
}

# Inflation Parameters
USA_inflation_mean = 4
USA_inflation_std = 1
India_inflation_mean = 7.379
India_inflation_std = 4.878

# Living Expenses
Monthly_living = 200000
Monthly_living_currency = 'INR'
Unexpected_expense_min_percent = 0.5
Unexpected_expense_max_percent = 15

# Post-Retirement Job Income
job_income_after_ret_annum = 2000000
job_yoy_increment_percent = 3
job_years_after_ret = 4
job_parameter_curency = 'INR'

# Tax Brackets - US Federal
US_federal_brackets = [
    (0, 22000, 0.10),
    (22000, 89450, 0.12),
    (89450, 190750, 0.22),
    (190750, 364200, 0.24),
    (364200, 462500, 0.32),
    (462500, 693750, 0.35),
    (693750, 1_000_000, 0.37)
]

# Tax Brackets - US State (California)
US_state_brackets = [
    (0, 20198, 0.01),
    (20198, 47884, 0.02),
    (47884, 75576, 0.04),
    (75576, 104910, 0.06),
    (104910, 132590, 0.08),
    (132590, 677278, 0.093),
    (677278, 812728, 0.103),
    (812728, 1354550, 0.113),
    (1354550, 1_000_000, 0.123)
]

# Tax Brackets - India
India_federal_brackets = [
    (0, 250000, 0.0),
    (250000, 500000, 0.05),
    (500000, 1000000, 0.20),
    (1000000, 10000000, 0.30)
]

# Fallback tax rates (not used with progressive brackets, kept for reference)
effective_tax_usa = 32
effective_tax_ind = 30

# Currency Exchange
default_USD_to_INR = 83.5
USD_INR_comission_percent = 0.550
USD_INR_mean_growth_percent = 3.040
USD_INR_STD = 5.57

# Fetch live exchange rate
try:
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    USD_INR_rate = data["rates"]["INR"]
    print(f"✓ Live USD to INR rate fetched: {USD_INR_rate:.2f}")
except:
    USD_INR_rate = default_USD_to_INR
    print(f'⚠ Unable to reach server, using default USD to INR: {USD_INR_rate:.2f}')

# Simulation Parameters
simulation_counts = 1000
realistic_target_survival_age = 100

# Derived Parameters
current_year = datetime.now().year
target_survival_year = year_of_birth + Age_diff_spouse + 100
years_to_calculate = target_survival_year - current_year

print(f"\\n📊 Simulation Parameters:")
print(f"   - Current Year: {current_year}")
print(f"   - Years to Simulate: {years_to_calculate}")
print(f"   - Retirement Year: {retirement_year}")
print(f"   - Number of Simulations: {simulation_counts}")


✓ Live USD to INR rate fetched: 89.75
\n📊 Simulation Parameters:
   - Current Year: 2025
   - Years to Simulate: 71
   - Retirement Year: 2028
   - Number of Simulations: 1000


In [ ]:
# HELPER FUNCTIONS


def calculate_mortgage_payment(principal, annual_rate, term_years):
    """Calculate monthly mortgage payment"""
    monthly_rate = annual_rate / 12 / 100
    payments = term_years * 12
    return principal * monthly_rate / (1 - (1 + monthly_rate) ** -payments)

def adjust_rand(generated_values, target_mean, target_std):
    """Adjust randomly generated values to match target mean and std"""
    current_mean = np.mean(generated_values)
    current_std = np.std(generated_values)
    if current_std == 0:
        return np.full_like(generated_values, target_mean)
    return (generated_values - current_mean) / current_std * target_std + target_mean

def calculate_progressive_tax(income, brackets):
    """Calculate tax based on progressive tax brackets"""
    if income <= 0:
        return 0
    
    tax = 0
    for bracket_start, bracket_end, rate in brackets:
        if income <= bracket_start:
            break
        
        taxable_in_bracket = min(income, bracket_end) - bracket_start
        if taxable_in_bracket > 0:
            tax += taxable_in_bracket * rate
        
        if income <= bracket_end:
            break
    
    return tax

def calculate_effective_tax_rate(income, brackets):
    """Calculate effective tax rate from progressive brackets"""
    if income <= 0:
        return 0
    tax = calculate_progressive_tax(income, brackets)
    return (tax / income) * 100

def calculate_social_security_benefit(year_of_birth, withdrawal_age, earnings_history, 
                                     future_earnings, current_year, retirement_year_param):
    """
    Calculate Social Security benefit based on actual earnings history.
    Uses SSA methodology with wage indexing and bend points.
    
    Parameters:
    - year_of_birth: Birth year
    - withdrawal_age: Age to start claiming (62-70)
    - earnings_history: Dict of {year: earnings}
    - future_earnings: Assumed earnings for future years BEFORE retirement
    - current_year: Current simulation year
    - retirement_year_param: Year when you stop working (for zero earnings after)
    
    Returns:
    - Monthly benefit amount at specified withdrawal age
    """
    
    # SSA wage index factors (approximate values for 2024 base year)
    # These are used to index past earnings to equivalent 2024 dollars
    wage_index_factors = {
        2014: 1.255, 2015: 1.232, 2016: 1.211, 2017: 1.182,
        2018: 1.148, 2019: 1.121, 2020: 1.094, 2021: 1.059,
        2022: 1.023, 2023: 1.010, 2024: 1.000
    }
    
    # Build complete earnings record (35 years needed for calculation)
    indexed_earnings = []
    
    # Add historical earnings (indexed)
    for year, earnings in earnings_history.items():
        index_factor = wage_index_factors.get(year, 1.0)
        indexed_earnings.append(earnings * index_factor)
    
    # Add future earnings
    age_62_year = year_of_birth + 62
    
    # Fill in years from current year until SS claiming age
    # BUT: Only add earnings until retirement year, then zeros after
    for year in range(current_year, min(age_62_year, year_of_birth + 67)):  # Up to age 67 max
        if year not in earnings_history:
            if year < retirement_year_param:
                # Before retirement: use future earnings assumption
                indexed_earnings.append(future_earnings)
            else:
                # After retirement: zero earnings (or could use post-retirement job income)
                indexed_earnings.append(0)
    
    # Pad with zeros if less than 35 years
    while len(indexed_earnings) < 35:
        indexed_earnings.append(0)
    
    # Take highest 35 years
    top_35_earnings = sorted(indexed_earnings, reverse=True)[:35]
    
    # Calculate Average Indexed Monthly Earnings (AIME)
    aime = sum(top_35_earnings) / 35 / 12
    
    # Apply bend points to calculate Primary Insurance Amount (PIA)
    # 2024 bend points: $1,174 and $7,078
    bend_point_1 = 1174
    bend_point_2 = 7078
    
    if aime <= bend_point_1:
        pia = aime * 0.90
    elif aime <= bend_point_2:
        pia = (bend_point_1 * 0.90) + ((aime - bend_point_1) * 0.32)
    else:
        pia = (bend_point_1 * 0.90) + ((bend_point_2 - bend_point_1) * 0.32) + ((aime - bend_point_2) * 0.15)
    
    # Adjust for early/late retirement
    # Full Retirement Age (FRA) for someone born in 1992 is 67
    fra = 67
    
    if withdrawal_age < fra:
        # Early retirement reduction: ~6.67% per year before FRA (up to 3 years), then 5% per year
        months_early = (fra - withdrawal_age) * 12
        if months_early <= 36:
            reduction = months_early * (5/9) / 100  # 5/9 of 1% per month
        else:
            reduction = (36 * (5/9) / 100) + ((months_early - 36) * (5/12) / 100)
        pia = pia * (1 - reduction)
    elif withdrawal_age > fra:
        # Delayed retirement credits: 8% per year after FRA
        years_delayed = withdrawal_age - fra
        pia = pia * (1 + 0.08 * years_delayed)
    
    return pia

def calculate_rmd(age, account_balance):
    """Calculate Required Minimum Distribution based on IRS life expectancy tables"""
    # IRS Uniform Lifetime Table (simplified version)
    rmd_factors = {
        73: 26.5, 74: 25.5, 75: 24.6, 76: 23.7, 77: 22.9, 78: 22.0,
        79: 21.1, 80: 20.2, 81: 19.4, 82: 18.5, 83: 17.7, 84: 16.8,
        85: 16.0, 86: 15.2, 87: 14.4, 88: 13.7, 89: 12.9, 90: 12.2,
        91: 11.5, 92: 10.8, 93: 10.1, 94: 9.5, 95: 8.9, 96: 8.4,
        97: 7.8, 98: 7.3, 99: 6.8, 100: 6.4
    }
    
    if age < 73:
        return 0
    elif age >= 100:
        factor = 6.4
    else:
        factor = rmd_factors.get(age, 6.4)
    
    return account_balance / factor

def apply_currency_conversion_fee(amount, commission_percent):
    """Apply currency conversion commission"""
    return amount * (1 - commission_percent / 100)

print("✓ Helper functions loaded")


✓ Helper functions loaded


In [ ]:
# SIMULATION FUNCTIONS


def gen_simulation_variables():
    """
    Generate random variables for simulation:
    - USD to INR growth
    - India inflation
    - USA inflation
    - Unexpected expenses
    - Social Security COLA adjustments
    - Investment YOY growth (including rents)
    """
    rand_var_df = pd.DataFrame()
    rand_var_df['USD_INR_growth'] = adjust_rand(
        np.random.normal(loc=USD_INR_mean_growth_percent, scale=USD_INR_STD, size=years_to_calculate),
        USD_INR_mean_growth_percent, USD_INR_STD
    )
    rand_var_df['IND_Inflation_growth'] = adjust_rand(
        np.random.normal(loc=India_inflation_mean, scale=India_inflation_std, size=years_to_calculate),
        India_inflation_mean, India_inflation_std
    )
    rand_var_df['USA_Inflation_growth'] = adjust_rand(
        np.random.normal(loc=USA_inflation_mean, scale=USA_inflation_std, size=years_to_calculate),
        USA_inflation_mean, USA_inflation_std
    )
    rand_var_df['unexpected_expense_rate'] = np.random.uniform(
        Unexpected_expense_min_percent, Unexpected_expense_max_percent, size=years_to_calculate
    )
    rand_var_df['social_sec_cola'] = adjust_rand(
        np.random.normal(loc=social_sec['avg_cola_yoy_growth'], scale=social_sec['avg_cola_std'], size=years_to_calculate),
        social_sec['avg_cola_yoy_growth'], social_sec['avg_cola_std']
    )

    for key, invs in Investments.items():
        means = invs.get('mean_yoy_growthrate_percent', 5)
        std_dev = invs.get('std_yoy_growthrate', 0)

        rand_var_df[f'{key}_growth'] = adjust_rand(
            np.random.normal(loc=means, scale=std_dev, size=years_to_calculate),
            means, std_dev
        )

        if invs.get('type') == 'real_estate':
            rand_var_df[f'{key}_rent_growth'] = adjust_rand(
                np.random.normal(loc=invs['rental_income_yoy_inc_percent'], scale=0, size=years_to_calculate),
                invs['rental_income_yoy_inc_percent'], 0
            )

    return rand_var_df


def simulate_retirement(simulation_var_df):
    """
    Perform yearly retirement simulation:
    - Adjust for inflation, unexpected expenses, mortgage costs
    - Calculate Social Security benefits based on actual earnings
    - Withdraw from investments if needed
    - Track if/when the money runs out
    """
    simulation_var_df['year'] = current_year + simulation_var_df.index
    simulation_var_df['age'] = simulation_var_df['year'] - year_of_birth

    # Calculate Social Security benefit once (will be inflated with COLA each year)
    base_ss_monthly_benefit = calculate_social_security_benefit(
        year_of_birth, 
        social_sec['withdrawl_age'],
        social_sec['earnings_history'],
        social_sec['future_earnings_assumption'],
        current_year,
        retirement_year
    )
    
    # Initialize columns with proper float dtype to avoid pandas warnings
    simulation_var_df['social_sec_income'] = 0.0
    simulation_var_df['rmd'] = 0.0
    simulation_var_df['effective_tax_rate'] = 0.0
    simulation_var_df['living_expense'] = 0.0
    simulation_var_df['unexpected_expense'] = 0.0
    simulation_var_df['ret_job_inc'] = 0.0
    simulation_var_df['post_tax_living_USD_needed'] = 0.0
    simulation_var_df['post_tax_all_mortgage'] = 0.0
    simulation_var_df['post_tax_net_expense'] = 0.0
    simulation_var_df['Pre_tax_net_expense'] = 0.0
    simulation_var_df['Pre_tax_expense_left'] = 0.0
    simulation_var_df['stock_current_value'] = 0.0
    simulation_var_df['stock_ret_current_value'] = 0.0
    simulation_var_df['cash_cert_current_value'] = 0.0
    simulation_var_df['Net_liquid_left'] = 0.0
    simulation_var_df['usd_inr_rate'] = 0.0
    simulation_var_df['inflated_living'] = 0.0
    
    for sim_year in simulation_var_df.index:
        # USD to INR rate calculation
        if sim_year == 0:
            simulation_var_df.loc[sim_year, 'usd_inr_rate'] = USD_INR_rate
        else:
            simulation_var_df.loc[sim_year, 'usd_inr_rate'] = simulation_var_df.loc[sim_year - 1, 'usd_inr_rate'] * (
                1 + simulation_var_df.loc[sim_year, 'USD_INR_growth'] / 100
            )

        # Inflation adjustments for living expenses
        if sim_year == 0:
            simulation_var_df.loc[sim_year, 'inflated_living'] = Monthly_living
        else:
            simulation_var_df.loc[sim_year, 'inflated_living'] = (
                simulation_var_df.loc[sim_year - 1, 'inflated_living'] *
                (1 + simulation_var_df.loc[sim_year - 1, 'IND_Inflation_growth'] / 100)
            )

        # Living and unexpected expenses post-retirement
        if simulation_var_df.loc[sim_year, 'year'] < retirement_year:
            simulation_var_df.loc[sim_year, 'living_expense'] = 0
            simulation_var_df.loc[sim_year, 'unexpected_expense'] = 0
        else:
            simulation_var_df.loc[sim_year, 'living_expense'] = simulation_var_df.loc[sim_year, 'inflated_living'] * 12
            simulation_var_df.loc[sim_year, 'unexpected_expense'] = (
                simulation_var_df.loc[sim_year, 'living_expense'] *
                (simulation_var_df.loc[sim_year, 'unexpected_expense_rate'] / 100)
            )

        # Post-retirement job income
        if (simulation_var_df.loc[sim_year, 'year'] < retirement_year) or \
           (simulation_var_df.loc[sim_year, 'year'] > retirement_year + job_years_after_ret):
            simulation_var_df.loc[sim_year, 'ret_job_inc'] = 0
        elif simulation_var_df.loc[sim_year, 'year'] == retirement_year:
            simulation_var_df.loc[sim_year, 'ret_job_inc'] = job_income_after_ret_annum
        else:
            simulation_var_df.loc[sim_year, 'ret_job_inc'] = simulation_var_df.loc[sim_year - 1, 'ret_job_inc'] * (
                1 + job_yoy_increment_percent / 100
            )

        # Net living (INR) to USD
        simulation_var_df.loc[sim_year, 'post_tax_living_USD_needed'] = (
            (simulation_var_df.loc[sim_year, 'living_expense'] +
             simulation_var_df.loc[sim_year, 'unexpected_expense'] -
             simulation_var_df.loc[sim_year, 'ret_job_inc']) /
            simulation_var_df.loc[sim_year, 'usd_inr_rate']
        )

        # Calculate real estate costs (mortgages) if any
        simulation_var_df.loc[sim_year, 'post_tax_all_mortgage'] = 0

        for key, invs in Investments.items():
            if invs.get('type') == 'real_estate':
                # Real estate logic would go here
                pass

        simulation_var_df.loc[sim_year, 'post_tax_net_expense'] = (
            simulation_var_df.loc[sim_year, 'post_tax_all_mortgage'] +
            simulation_var_df.loc[sim_year, 'post_tax_living_USD_needed']
        )

        # Pre-tax expense calculation
        if simulation_var_df.loc[sim_year, 'year'] <= retirement_year + 2:
            if effective_tax_usa != 0:
                simulation_var_df.loc[sim_year, 'Pre_tax_net_expense'] = (
                    simulation_var_df.loc[sim_year, 'post_tax_net_expense'] / (1 - effective_tax_usa / 100)
                )
                simulation_var_df.loc[sim_year, 'effective_tax_rate'] = effective_tax_usa
            else:
                simulation_var_df.loc[sim_year, 'Pre_tax_net_expense'] = simulation_var_df.loc[sim_year, 'post_tax_net_expense']
                simulation_var_df.loc[sim_year, 'effective_tax_rate'] = 0
        else:
            if effective_tax_ind != 0:
                simulation_var_df.loc[sim_year, 'Pre_tax_net_expense'] = (
                    simulation_var_df.loc[sim_year, 'post_tax_net_expense'] / (1 - effective_tax_ind / 100)
                )
                simulation_var_df.loc[sim_year, 'effective_tax_rate'] = effective_tax_ind
            else:
                simulation_var_df.loc[sim_year, 'Pre_tax_net_expense'] = simulation_var_df.loc[sim_year, 'post_tax_net_expense']
                simulation_var_df.loc[sim_year, 'effective_tax_rate'] = 0

        # Investment growth and tracking
        simulation_var_df.loc[sim_year, 'stock_current_value'] = 0
        simulation_var_df.loc[sim_year, 'stock_ret_current_value'] = 0
        simulation_var_df.loc[sim_year, 'cash_cert_current_value'] = 0

        for key, invsts in Investments.items():
            if invsts['type'] in ['stock', 'stock_retirement', 'cash_certificate']:
                if sim_year == 0:
                    simulation_var_df.loc[sim_year, f'{key}_current_value'] = invsts['current_value']
                else:
                    simulation_var_df.loc[sim_year, f'{key}_current_value'] = (
                        simulation_var_df.loc[sim_year - 1, f'{key}_current_value'] *
                        (1 + (simulation_var_df.loc[sim_year - 1, f'{key}_growth'] / 100))
                    )

                if invsts['type'] == 'stock':
                    simulation_var_df.loc[sim_year, 'stock_current_value'] += simulation_var_df.loc[sim_year, f'{key}_current_value']
                elif invsts['type'] == 'stock_retirement':
                    simulation_var_df.loc[sim_year, 'stock_ret_current_value'] += simulation_var_df.loc[sim_year, f'{key}_current_value']
                elif invsts['type'] == 'cash_certificate':
                    simulation_var_df.loc[sim_year, 'cash_cert_current_value'] += simulation_var_df.loc[sim_year, f'{key}_current_value']

        # Calculate RMDs for retirement accounts
        if simulation_var_df.loc[sim_year, 'age'] >= 73:
            simulation_var_df.loc[sim_year, 'rmd'] = calculate_rmd(
                simulation_var_df.loc[sim_year, 'age'],
                simulation_var_df.loc[sim_year, 'stock_ret_current_value']
            )
        
        # Calculate withdrawals needed
        simulation_var_df.loc[sim_year, 'Pre_tax_expense_left'] = simulation_var_df.loc[sim_year, 'Pre_tax_net_expense'] * 1

        # IMPROVED SOCIAL SECURITY CALCULATION WITH COLA
        if simulation_var_df.loc[sim_year, 'age'] >= social_sec['withdrawl_age']:
            if sim_year == 0 or simulation_var_df.loc[sim_year, 'age'] == social_sec['withdrawl_age']:
                # First year of receiving SS
                simulation_var_df.loc[sim_year, 'social_sec_income'] = base_ss_monthly_benefit * 12
            else:
                # Apply COLA from previous year
                simulation_var_df.loc[sim_year, 'social_sec_income'] = (
                    simulation_var_df.loc[sim_year - 1, 'social_sec_income'] *
                    (1 + simulation_var_df.loc[sim_year - 1, 'social_sec_cola'] / 100)
                )
            
            simulation_var_df.loc[sim_year, 'Pre_tax_expense_left'] -= simulation_var_df.loc[sim_year, 'social_sec_income']

        inv_rat = {}

        # Withdraw from regular stocks first
        if (simulation_var_df.loc[sim_year, 'Pre_tax_expense_left'] > 0) and (simulation_var_df.loc[sim_year, 'stock_current_value'] > 0):
            for key, invsts in Investments.items():
                if invsts['type'] == 'stock':
                    inv_rat[f'{key}'] = (simulation_var_df.loc[sim_year, f'{key}_current_value'] /
                                         simulation_var_df.loc[sim_year, 'stock_current_value'])

            if simulation_var_df.loc[sim_year, 'Pre_tax_expense_left'] <= simulation_var_df.loc[sim_year, 'stock_current_value']:
                simulation_var_df.loc[sim_year, 'stock_current_value'] -= simulation_var_df.loc[sim_year, 'Pre_tax_expense_left']
                simulation_var_df.loc[sim_year, 'Pre_tax_expense_left'] = 0
            else:
                simulation_var_df.loc[sim_year, 'Pre_tax_expense_left'] -= simulation_var_df.loc[sim_year, 'stock_current_value']
                simulation_var_df.loc[sim_year, 'stock_current_value'] = 0

            for key, invsts in Investments.items():
                if invsts['type'] == 'stock':
                    simulation_var_df.loc[sim_year, f'{key}_current_value'] = (
                        inv_rat[f'{key}'] * simulation_var_df.loc[sim_year, 'stock_current_value']
                    )

        # Withdraw from retirement accounts (with penalty if age < 59.5)
        if (simulation_var_df.loc[sim_year, 'Pre_tax_expense_left'] > 0) and (simulation_var_df.loc[sim_year, 'stock_ret_current_value'] > 0):
            penalty_withdraw_multiplier = 1.1 if simulation_var_df.loc[sim_year, 'age'] < 59.5 else 1.0

            for key, invsts in Investments.items():
                if invsts['type'] == 'stock_retirement':
                    inv_rat[f'{key}'] = (
                        simulation_var_df.loc[sim_year, f'{key}_current_value'] /
                        simulation_var_df.loc[sim_year, 'stock_ret_current_value']
                    )

            required_amount = simulation_var_df.loc[sim_year, 'Pre_tax_expense_left'] * penalty_withdraw_multiplier
            if required_amount <= simulation_var_df.loc[sim_year, 'stock_ret_current_value']:
                simulation_var_df.loc[sim_year, 'stock_ret_current_value'] -= required_amount
                simulation_var_df.loc[sim_year, 'Pre_tax_expense_left'] = 0
            else:
                simulation_var_df.loc[sim_year, 'Pre_tax_expense_left'] -= (
                    simulation_var_df.loc[sim_year, 'stock_ret_current_value'] / penalty_withdraw_multiplier
                )
                simulation_var_df.loc[sim_year, 'stock_ret_current_value'] = 0

            for key, invsts in Investments.items():
                if invsts['type'] == 'stock_retirement':
                    simulation_var_df.loc[sim_year, f'{key}_current_value'] = (
                        inv_rat[f'{key}'] * simulation_var_df.loc[sim_year, 'stock_ret_current_value']
                    )

        # Withdraw from cash certificates last
        if (simulation_var_df.loc[sim_year, 'Pre_tax_expense_left'] > 0) and (simulation_var_df.loc[sim_year, 'cash_cert_current_value'] > 0):
            for key, invsts in Investments.items():
                if invsts['type'] == 'cash_certificate':
                    inv_rat[f'{key}'] = (
                        simulation_var_df.loc[sim_year, f'{key}_current_value'] /
                        simulation_var_df.loc[sim_year, 'cash_cert_current_value']
                    )

            if simulation_var_df.loc[sim_year, 'Pre_tax_expense_left'] <= simulation_var_df.loc[sim_year, 'cash_cert_current_value']:
                simulation_var_df.loc[sim_year, 'cash_cert_current_value'] -= simulation_var_df.loc[sim_year, 'Pre_tax_expense_left']
                simulation_var_df.loc[sim_year, 'Pre_tax_expense_left'] = 0
            else:
                simulation_var_df.loc[sim_year, 'Pre_tax_expense_left'] -= simulation_var_df.loc[sim_year, 'cash_cert_current_value']
                simulation_var_df.loc[sim_year, 'cash_cert_current_value'] = 0

            for key, invsts in Investments.items():
                if invsts['type'] == 'cash_certificate':
                    simulation_var_df.loc[sim_year, f'{key}_current_value'] = (
                        inv_rat[f'{key}'] * simulation_var_df.loc[sim_year, 'cash_cert_current_value']
                    )

        # Check net liquidity left
        simulation_var_df.loc[sim_year, 'Net_liquid_left'] = (
            simulation_var_df.loc[sim_year, 'stock_current_value'] +
            simulation_var_df.loc[sim_year, 'stock_ret_current_value'] +
            simulation_var_df.loc[sim_year, 'cash_cert_current_value']
        )

        if simulation_var_df.loc[sim_year, 'Net_liquid_left'] <= 0:
            return simulation_var_df.loc[sim_year, 'age'], simulation_var_df

    return simulation_var_df.loc[sim_year, 'age'], simulation_var_df

print("✓ Simulation functions loaded")


✓ Simulation functions loaded


In [ ]:
# TEST SOCIAL SECURITY CALCULATION

print('='*80)
print('SOCIAL SECURITY BENEFIT CALCULATION (Based on Actual Earnings)')
print('='*80)

# Test at different withdrawal ages
test_ages = [62, 65, 67, 70]
current_test_year = datetime.now().year

print(f"\nYour Earnings History:")
print(f"{'Year':<10} {'Social Security Wages':<25}")
print('-' * 40)
for year, earnings in sorted(actual_earnings_history.items()):
    print(f"{year:<10} ${earnings:>20,}")

print(f"\nFuture years (until retirement) will assume: ${social_sec['future_earnings_assumption']:,}")

print(f"\n\nEstimated Monthly Social Security Benefits:")
print(f"{'Withdrawal Age':<20} {'Monthly Benefit':<20} {'Annual Benefit':<20}")
print('-' * 60)

for age in test_ages:
    monthly_benefit = calculate_social_security_benefit(
        year_of_birth,
        age,
        actual_earnings_history,
        social_sec['future_earnings_assumption'],
        current_test_year,
        retirement_year
    )
    annual_benefit = monthly_benefit * 12
    
    label = f"Age {age}"
    if age == 67:
        label += " (FRA)"
    elif age == 62:
        label += " (Early)"
    elif age == 70:
        label += " (Delayed)"
    
    print(f"{label:<20} ${monthly_benefit:>17,.2f}  ${annual_benefit:>17,.2f}")

print(f"\n\nYour current configuration uses withdrawal age: {social_sec['withdrawl_age']}")
selected_monthly = calculate_social_security_benefit(
    year_of_birth,
    social_sec['withdrawl_age'],
    actual_earnings_history,
    social_sec['future_earnings_assumption'],
    current_test_year,
    retirement_year
)
selected_annual = selected_monthly * 12

print(f"\n💡 Important Note:")
print(f"   You plan to retire at age {retirement_year - year_of_birth} (year {retirement_year})")
print(f"   But you won't claim Social Security until age {social_sec['withdrawl_age']}")
print(f"   The calculation assumes $168,600 earnings through {retirement_year}, then $0 after.")
print(f"   This gives you only {retirement_year - 2014} years of substantial earnings for SS calculation.")

print(f"Expected monthly benefit at age {social_sec['withdrawl_age']}: ${selected_monthly:,.2f}")
print(f"Expected annual benefit at age {social_sec['withdrawl_age']}: ${selected_annual:,.2f}")
print(f"\nNote: Benefits will increase with COLA (avg {social_sec['avg_cola_yoy_growth']}% per year)")
print('='*80)


SOCIAL SECURITY BENEFIT CALCULATION (Based on Actual Earnings)

Your Earnings History:
Year       Social Security Wages    
----------------------------------------
2014       $                  37
2015       $                   0
2016       $                   0
2017       $              24,022
2018       $             123,623
2019       $             121,380
2020       $             137,700
2021       $             142,800
2022       $             133,629
2023       $             160,200
2024       $             168,600

Future years (until retirement) will assume: $168,600


Estimated Monthly Social Security Benefits:
Withdrawal Age       Monthly Benefit      Annual Benefit      
------------------------------------------------------------
Age 62 (Early)       $         2,769.00  $        33,227.99
Age 65               $         3,428.28  $        41,139.42
Age 67 (FRA)         $         3,955.71  $        47,468.56
Age 70 (Delayed)     $         4,905.08  $        58,861.01


Your 

In [26]:
# RUN MONTE CARLO SIMULATIONS

print(f"🚀 Starting {simulation_counts} Monte Carlo simulations...")
print(f"   This may take a few minutes...\\n")

# Storage for all simulations - we need full DataFrames for worst-case analysis
all_simulations = []  # List of (broke_age, simulation_df, simulation_id) tuples
broke_ages = []
net_liq_results = []

# Run simulations
for i in range(simulation_counts):
    if (i + 1) % 100 == 0:
        print(f"   Progress: {i+1}/{simulation_counts} simulations complete...")
    
    rand_var_df = gen_simulation_variables()
    broke_age, simulation_var_df = simulate_retirement(rand_var_df.copy())
    
    # Store results
    broke_ages.append(broke_age)
    net_liq_results.append(simulation_var_df['Net_liquid_left'].rename(i))
    all_simulations.append((broke_age, simulation_var_df.copy(), i))

# Combine net liquid results
net_liq = pd.concat(net_liq_results, axis=1)
net_liq['age'] = simulation_var_df['age']

# Sort simulations by broke age for easy access to worst/best cases
all_simulations_sorted = sorted(all_simulations, key=lambda x: x[0])

# Identify key scenarios
worst_case = all_simulations_sorted[0]
p10_case = all_simulations_sorted[int(len(all_simulations_sorted) * 0.1)]
p25_case = all_simulations_sorted[int(len(all_simulations_sorted) * 0.25)]
median_case = all_simulations_sorted[len(all_simulations_sorted) // 2]
p75_case = all_simulations_sorted[int(len(all_simulations_sorted) * 0.75)]
p90_case = all_simulations_sorted[int(len(all_simulations_sorted) * 0.90)]
best_case = all_simulations_sorted[-1]

print(f"\\n✓ All {simulation_counts} simulations complete!")
print(f"\\n📋 Key Scenarios Identified:")
print(f"   Worst Case (Sim #{worst_case[2]}): Broke at age {worst_case[0]:.1f}")
print(f"   10th Percentile (Sim #{p10_case[2]}): Broke at age {p10_case[0]:.1f}")
print(f"   25th Percentile (Sim #{p25_case[2]}): Broke at age {p25_case[0]:.1f}")
print(f"   Median (Sim #{median_case[2]}): Broke at age {median_case[0]:.1f}")
print(f"   75th Percentile (Sim #{p75_case[2]}): Broke at age {p75_case[0]:.1f}")
print(f"   90th Percentile (Sim #{p90_case[2]}): Broke at age {p90_case[0]:.1f}")
print(f"   Best Case (Sim #{best_case[2]}): Broke at age {best_case[0]:.1f}")


🚀 Starting 1000 Monte Carlo simulations...
   This may take a few minutes...\n
   Progress: 100/1000 simulations complete...
   Progress: 200/1000 simulations complete...
   Progress: 300/1000 simulations complete...
   Progress: 400/1000 simulations complete...
   Progress: 500/1000 simulations complete...
   Progress: 600/1000 simulations complete...
   Progress: 700/1000 simulations complete...
   Progress: 800/1000 simulations complete...
   Progress: 900/1000 simulations complete...
   Progress: 1000/1000 simulations complete...
\n✓ All 1000 simulations complete!
\n📋 Key Scenarios Identified:
   Worst Case (Sim #299): Broke at age 36.0
   10th Percentile (Sim #892): Broke at age 92.0
   25th Percentile (Sim #163): Broke at age 103.0
   Median (Sim #440): Broke at age 103.0
   75th Percentile (Sim #723): Broke at age 103.0
   90th Percentile (Sim #894): Broke at age 103.0
   Best Case (Sim #999): Broke at age 103.0


In [27]:
# SUMMARY STATISTICS

print('='*80)
print('RETIREMENT SIMULATION RESULTS')
print('='*80)
print(f'Number of simulations: {simulation_counts}')
print(f'Target survival age: {realistic_target_survival_age}')
print()
print('BROKE AGE STATISTICS:')
print(f'  Minimum broke age:    {np.min(broke_ages):.1f}')
print(f'  10th percentile:      {np.percentile(broke_ages, 10):.1f}')
print(f'  25th percentile:      {np.percentile(broke_ages, 25):.1f}')
print(f'  Median (50th %ile):   {np.median(broke_ages):.1f}')
print(f'  Mean broke age:       {np.mean(broke_ages):.1f}')
print(f'  75th percentile:      {np.percentile(broke_ages, 75):.1f}')
print(f'  90th percentile:      {np.percentile(broke_ages, 90):.1f}')
print(f'  Maximum broke age:    {np.max(broke_ages):.1f}')
print(f'  Std deviation:        {np.std(broke_ages):.2f}')
print()
failure_rate = sum(1 for age in broke_ages if age < realistic_target_survival_age) * 100 / simulation_counts
print(f'FAILURE RATE: {failure_rate:.1f}%')
print(f'  (Percentage of simulations where money ran out before age {realistic_target_survival_age})')
print()
success_rate = 100 - failure_rate
print(f'SUCCESS RATE: {success_rate:.1f}%')
print('='*80)

# Show distribution plots
random_var_box = px.box(rand_var_df, title="Distribution of Random Variables")
random_var_box.show()

broke_age_hist = px.histogram(broke_ages, nbins=50, title="Distribution of Broke Ages",
                               labels={'value': 'Broke Age', 'count': 'Frequency'})
broke_age_hist.add_vline(x=realistic_target_survival_age, line_dash="dash", 
                          line_color="red", annotation_text="Target Age")
broke_age_hist.show()


RETIREMENT SIMULATION RESULTS
Number of simulations: 1000
Target survival age: 100

BROKE AGE STATISTICS:
  Minimum broke age:    36.0
  10th percentile:      91.8
  25th percentile:      103.0
  Median (50th %ile):   103.0
  Mean broke age:       98.1
  75th percentile:      103.0
  90th percentile:      103.0
  Maximum broke age:    103.0
  Std deviation:        15.31

FAILURE RATE: 10.5%
  (Percentage of simulations where money ran out before age 100)

SUCCESS RATE: 89.5%


In [28]:
# NET LIQUID ASSETS VISUALIZATION (Percentiles)

liq = go.Figure()

# Calculate percentiles across simulations
percentiles = [10, 25, 50, 75, 90]
colors = ['#EF5350', '#FF9800', '#FFD700', '#66BB6A', '#42A5F5']

for idx, p in enumerate(percentiles):
    percentile_values = net_liq.drop('age', axis=1).quantile(p/100, axis=1)
    liq.add_trace(go.Scatter(
        x=net_liq['age'], 
        y=percentile_values, 
        mode='lines', 
        name=f'{p}th percentile',
        line=dict(width=3 if p == 50 else 2, color=colors[idx])
    ))

# Add a few sample trajectories for context
sample_indices = [worst_case[2], p25_case[2], median_case[2], p75_case[2], best_case[2]]
for idx in sample_indices:
    if idx in net_liq.columns:
        liq.add_trace(go.Scatter(
            x=net_liq['age'], 
            y=net_liq[idx], 
            mode='lines', 
            name=f'Sample Sim #{idx}',
            line=dict(width=1, dash='dot'),
            opacity=0.4,
            showlegend=False
        ))

liq.update_layout(
    title='Net Liquid Assets Over Time (Percentiles)',
    xaxis_title='Age',
    yaxis_title='Net Liquid Value (USD)',
    template='plotly_dark',
    hovermode='x unified',
    height=600
)
liq.show()


In [29]:
# ================================================================================
# VARIABLES MOVEMENT ANALYSIS - Worst Case & Key Percentiles
# ================================================================================

# Define scenarios to visualize
scenarios = [
    ('Worst Case', worst_case, '#D32F2F'),
    ('10th Percentile', p10_case, '#F57C00'),
    ('Median (50th)', median_case, '#FFD600'),
    ('90th Percentile', p90_case, '#388E3C'),
]

# Create subplots for key variables
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Net Liquid Assets',
        'Pre-tax Net Expenses', 
        'Social Security Income',
        'Required Minimum Distributions (RMDs)',
        'Effective Tax Rate',
        'Investment Breakdown'
    ),
    specs=[[{'secondary_y': False}, {'secondary_y': False}],
           [{'secondary_y': False}, {'secondary_y': False}],
           [{'secondary_y': False}, {'secondary_y': False}]],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# Plot each scenario
for scenario_name, scenario_data, color in scenarios:
    broke_age, df, sim_id = scenario_data
    
    # 1. Net Liquid Assets
    fig.add_trace(
        go.Scatter(x=df['age'], y=df['Net_liquid_left'], 
                   name=scenario_name, line=dict(color=color, width=2),
                   legendgroup=scenario_name),
        row=1, col=1
    )
    
    # 2. Pre-tax Net Expenses
    fig.add_trace(
        go.Scatter(x=df['age'], y=df['Pre_tax_net_expense'], 
                   name=scenario_name, line=dict(color=color, width=2),
                   legendgroup=scenario_name, showlegend=False),
        row=1, col=2
    )
    
    # 3. Social Security Income
    fig.add_trace(
        go.Scatter(x=df['age'], y=df['social_sec_income'], 
                   name=scenario_name, line=dict(color=color, width=2),
                   legendgroup=scenario_name, showlegend=False),
        row=2, col=1
    )
    
    # 4. RMDs
    fig.add_trace(
        go.Scatter(x=df['age'], y=df['rmd'], 
                   name=scenario_name, line=dict(color=color, width=2),
                   legendgroup=scenario_name, showlegend=False),
        row=2, col=2
    )
    
    # 5. Effective Tax Rate
    fig.add_trace(
        go.Scatter(x=df['age'], y=df['effective_tax_rate'], 
                   name=scenario_name, line=dict(color=color, width=2),
                   legendgroup=scenario_name, showlegend=False),
        row=3, col=1
    )
    
    # 6. Investment Breakdown (stacked area for one scenario - median)
    if scenario_name == 'Median (50th)':
        # Get investment columns
        inv_cols = [col for col in df.columns if col.endswith('_current_value')]
        if inv_cols:
            for inv_col in inv_cols:
                inv_name = inv_col.replace('_current_value', '')
                fig.add_trace(
                    go.Scatter(x=df['age'], y=df[inv_col], 
                               name=inv_name, 
                               stackgroup='one',
                               mode='lines'),
                    row=3, col=2
                )

# Update axes labels
fig.update_xaxes(title_text="Age", row=3, col=1)
fig.update_xaxes(title_text="Age", row=3, col=2)
fig.update_yaxes(title_text="USD", row=1, col=1)
fig.update_yaxes(title_text="USD", row=1, col=2)
fig.update_yaxes(title_text="USD/year", row=2, col=1)
fig.update_yaxes(title_text="USD", row=2, col=2)
fig.update_yaxes(title_text="%", row=3, col=1)
fig.update_yaxes(title_text="USD", row=3, col=2)

# Update layout
fig.update_layout(
    title_text="Detailed Variables Movement Analysis: Worst Case vs. Key Percentiles",
    height=1200,
    template='plotly_dark',
    hovermode='x unified',
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()

# Print detailed analysis for worst case
print("\\n" + "="*80)
print("WORST CASE SCENARIO ANALYSIS")
print("="*80)
worst_broke_age, worst_df, worst_sim_id = worst_case
print(f"Simulation ID: #{worst_sim_id}")
print(f"Broke at Age: {worst_broke_age:.1f}")
print(f"\\nKey Metrics at Retirement (Age ~{retirement_year - year_of_birth}):")
ret_year_idx = retirement_year - current_year
if ret_year_idx < len(worst_df):
    ret_data = worst_df.iloc[ret_year_idx]
    print(f"  Net Liquid Assets: ${ret_data['Net_liquid_left']:,.0f}")
    print(f"  Annual Expenses (pre-tax): ${ret_data['Pre_tax_net_expense']:,.0f}")
    
print(f"\\nKey Metrics at Broke Age ({worst_broke_age:.0f}):")
if len(worst_df) > 0:
    final_data = worst_df.iloc[-1]
    print(f"  Final Net Liquid: ${final_data['Net_liquid_left']:,.0f}")
    print(f"  Social Security Income: ${final_data['social_sec_income']:,.0f}/year")
    print(f"  Effective Tax Rate: {final_data['effective_tax_rate']:.1f}%")
    
print("="*80)


\n================================================================================
WORST CASE SCENARIO ANALYSIS
Simulation ID: #299
Broke at Age: 36.0
\nKey Metrics at Retirement (Age ~36):
  Net Liquid Assets: $-313,167
  Annual Expenses (pre-tax): $29,176
\nKey Metrics at Broke Age (36):
  Final Net Liquid: $0
  Social Security Income: $0/year
  Effective Tax Rate: 0.0%
